In [ ]:
from bokeh.plotting import figure, show, output_file
from bokeh.palettes import Spectral11, Greys9, Spectral6, Inferno6,Viridis10
import pandas as pd
import numpy as np
from bokeh.io import export_png
from colorcolor import compute_colors as colcol 
from bokeh.layouts import row,column
import pickle as pk
from sqlalchemy import *
import os
from itertools import combinations as comb
from bokeh.plotting import ColumnDataSource 
from bokeh.models import HoverTool

## Computing Colors in Large Parameter Spaces 

There are 9,210 models in our database. If you have 10 filter options, the parameter space immediate gets large. For this analysis I chose to first compute a large dataframe with all possible combinations photometric observations and color observations. E.G. flux at filter1 and color from filter1-filter2. Then I go through and analyze what they look like. 


The function below takes **FOREVER** to run. There is a parallelized version in `scripts/` that is **MUCH** faster. If you wish to run this next part, I'd suggest going there. Otherwise skip to the next block where I load recomputed results 

In [ ]:
#to compute all the colors in WFIRST make sure the ALBEDO_DB environment variable is set 

#df = colcol.WFIRST_colors()

## WFIRST Color Fig From Paper showing Color-Space (Fig 7)

This creates color-color figure 7 from the paper. Toggle between cloud_free =True/False and colorby,sizeby

There is an online version of this at natashabatalha.github.io/color-color.html 

And the script to create the online version is in `scripts/`

In [ ]:
#figure 7 has four panels. 2 cloud free, 2 with clouds, toggle this parameter here to which between cloud 
#and cloud free 
cloud_free = False
colorby = 'distance'
sizeby = 'metallicity'

#as I said before, let's load this in since its all precomputed 
allfluxes = pk.load(open('wfirst_colors_dataframe.pk','rb'))
et = allfluxes

everything=et.dropna()[~et.dropna().isin([np.inf, -np.inf])]

if cloud_free: everything = everything.loc[everything['cloud']==0] 
#color map 
col_map={}
alp = {}
siz_map={}
ic=9

#COLOR BY DISTANCE 
for i in colcol.print_available(colorby)[1]:
    col_map[str(i)] = Spectral11[ic]
    alp[str(i)] = 1.0
    ic=ic-1

#SIZE BY METALLICITY 
for i,s in zip(colcol.print_available(sizeby)[1], [10,15,20,25,30,35]):
    if cloud_free: 
        siz_map[str(i)] = s
    else:
        siz_map[str(i)] = 15
sizes = [siz_map[str(d)] for d in everything[sizeby]]

cols = [col_map[str(d)] for d in everything[colorby]]
alps = [alp[str(d)] for d in everything[colorby]]


cf1 = figure(x_axis_label='575-883', y_axis_label='506-575',
           plot_height=1400, plot_width=1000)#, x_range=[-8,-3], y_range=[-1.4,0.25])
cf1.circle(everything['575883'],everything['506575'],  color=cols,alpha=alps, size = sizes)


sizes = [siz_map[str(d)] for d in everything[everything['phase']==0][sizeby]]
cols = [col_map[str(d)] for d in everything.loc[everything['phase']==0][colorby]]
alps = [alp[str(d)] for d in everything.loc[everything['phase']==0][colorby]]
#'575883','506575',
cf = figure(x_axis_label='575-883', y_axis_label='506-575',
           plot_height=1400, plot_width=1000)
cf.circle(everything.loc[everything['phase']==0]['575883'], everything.loc[everything['phase']==0]['506575'], color=cols,alpha=alps, size = sizes)


#SO much formatting
cf.xaxis.axis_label_text_font='times'
cf.yaxis.axis_label_text_font='times'
cf.xaxis.major_label_text_font_size='55pt'
cf.yaxis.major_label_text_font_size='55pt'
cf.xaxis.axis_label_text_font_size='52pt'
cf.yaxis.axis_label_text_font_size='52pt'
cf.xaxis.major_label_text_font='times'
cf.yaxis.major_label_text_font='times'
cf.xaxis.axis_label_text_font_style = 'bold'
cf.yaxis.axis_label_text_font_style = 'bold'

cf1.xaxis.axis_label_text_font='times'
cf1.yaxis.axis_label_text_font='times'
cf1.xaxis.major_label_text_font_size='55pt'
cf1.yaxis.major_label_text_font_size='55pt'
cf1.xaxis.axis_label_text_font_size='52pt'
cf1.yaxis.axis_label_text_font_size='52pt'
cf1.xaxis.major_label_text_font='times'
cf1.yaxis.major_label_text_font='times'
cf1.xaxis.axis_label_text_font_style = 'bold'
cf1.yaxis.axis_label_text_font_style = 'bold'

show(row(cf,cf1))